In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
estados = ['California','New_Jersey', 'Florida', 'Illinois'] # Datasets usados para el estudio

In [3]:
ruta_google_review = '../datasets/raw/Google Maps' # Ruta en maps de Google

In [4]:
metadata_google = pd.read_csv('../datasets/raw/Google Maps/all_in_one_limpio_con_nombres.csv.gz',compression='gzip') # metadata google

In [5]:
metadata_google.drop(columns='name',inplace=True)

In [6]:
dic_estados = {}
for estado in estados:
    ruta_archivo = f'{ruta_google_review}/reviews-estados/review-{estado}/all_{estado}.parquet'
    review_estado = pd.read_parquet(ruta_archivo)
    dic_estados[estado] = review_estado

In [43]:
dic_estados['California']

,user_id,name,time,rating,text,pics,gmap_id,resp_time,resp_text
0,1.089912e+20,Song Ro,2021-01-06 05:12:07.056,5,Love there korean rice cake.,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,NaT,None
1,1.112903e+20,Rafa Robles,2021-02-09 05:47:28.663,5,Good very good,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,NaT,None
2,1.126404e+20,David Han,2020-03-08 05:04:42.296,4,They make Korean traditional food very properly.,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,NaT,None
3,1.174403e+20,Anthony Kim,2019-03-07 05:56:56.355,5,Short ribs are very delicious.,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,NaT,None
4,1.005808e+20,Mario Marzouk,2017-05-16 05:01:41.933,5,Great food and prices the portions are large,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,NaT,None
...,...,...,...,...,...,...,...,...,...
2699995,1.113423e+20,Byunguk Kim,2019-11-10 05:11:14.409,5,None,None,0x80c2bea30829f279:0x39aa953ee93734ed,NaT,None
2699996,1.080818e+20,Ceveda Craytonhooks,2020-02-02 02:00:53.411,5,Beautiful salon. The staff and atmosphere were...,None,0x8094675073616747:0x9f935a9b9046a9ba,NaT,None
2699997,1.175914e+20,Natacha Thompson,2018-06-10 02:30:06.165,5,Flo is a great beautician. She is very patie...,[{'url': ['https://lh5.googleusercontent.com/p...,0x8094675073616747:0x9f935a9b9046a9ba,2018-06-10 02:48:52.831000064,Thank you! It was my pleasure to give you exa...
2699998,1.124266e+20,Eleanor Aikins,2019-02-26 17:49:39.295,5,I Been going to have a variety of hair style a...,None,0x8094675073616747:0x9f935a9b9046a9ba,NaT,None


In [44]:
dic_estados['California']['user_id'].value_counts()

user_id
1.033885e+20    313
1.077740e+20    144
1.030183e+20    131
1.119374e+20    131
1.150273e+20    129
               ... 
1.084716e+20      1
1.154589e+20      1
1.128638e+20      1
1.158468e+20      1
1.122199e+20      1
Name: count, Length: 973518, dtype: int64

Borro las filas donde user_id time y gmap_id esten duplicados

In [7]:
dic_estados['California'].drop_duplicates(subset=['user_id','time','gmap_id'],inplace=True)

In [8]:
dic_estados['California']['year'] = dic_estados['California']['time'].dt.year
userAño_California = dic_estados['California'].groupby('year')['user_id'].count().reset_index()

In [9]:
df_full = pd.merge(dic_estados['California'],metadata_google,on='gmap_id',how='inner')
df_full['Estado_Perteneciente'].value_counts()

Estado_Perteneciente
Florida    2611999
Alabama     117174
Georgia         15
Name: count, dtype: int64

Al buscar las corrdenadas en maps de los lugares donde el state no era California, vi que si perecen a los lugares que estan en metadata-sitios, por esto hay que modificar las review, en este caso como estas dos no son usadas simplemente no las tomo.<br>
Y en reviews, voy a liminar los que no pertenzcan a ese estado.

In [53]:
df_full[df_full['Estado_Perteneciente'] != 'California'].sample()

,user_id,name_x,time,rating,text,pics,gmap_id,resp_time,resp_text,year,latitude,longitude,category,avg_rating,Estado_Perteneciente,name_y
2171420,1.113673e+20,Jon Armstrong (Cell),2016-09-10 16:47:01.819,5,Fast and easy smog check.,None,0x80d1393785330c7f:0xbb16326cef2e95c0,NaT,None,2016,33.610754,-114.593199,['Auto repair shop' 'Smog inspection station'],4.2,Arizona,EZ Lube


Arreglo el estado de california

In [55]:
df_full_California = df_full[df_full['Estado_Perteneciente'] == 'California']

In [56]:
review_California = dic_estados['California'][dic_estados['California']['gmap_id'].isin(df_full_California['gmap_id'])]

Veo si esos estados extra estan dentro de los seleccionados de ser asi concateno la informacion

In [67]:
for estado in dic_estados:
    if estado in df_full['Estado_Perteneciente'].values and estado not in df_full_California['Estado_Perteneciente'].values:
        dic_estados[estado] =  pd.concat(dic_estados[estado],df_full.iloc[:,:10])
        

Realizo el proceso para todos

In [79]:
for estado in dic_estados:
    dic_estados[estado].drop_duplicates(subset=['user_id','time','gmap_id'],inplace=True)
    dic_estados[estado]['year'] = dic_estados[estado]['time'].dt.year
    df_full = pd.merge(dic_estados[estado],metadata_google,on='gmap_id',how='inner')
    df_full_California = df_full[df_full['Estado_Perteneciente'] == estado]
    dic_estados[estado] = dic_estados[estado][dic_estados[estado]['gmap_id'].isin(df_full_California['gmap_id'])]
    for estado in dic_estados:
        if estado in df_full['Estado_Perteneciente'].values and estado not in df_full_California['Estado_Perteneciente'].values:
             dic_estados[estado] =  pd.concat([dic_estados[estado],df_full.iloc[:,:10]],ignore_index=True)

In [81]:
dic_estados['Florida']

,user_id,name,time,rating,text,pics,gmap_id,resp_time,resp_text,name_x,year
7,1.142782e+20,Mekyel'la Beauty Bar,2021-04-19 02:16:00.242,5,The Husband and Wife are so nice and helpful. ...,None,0x88c33f1c8e1f99a3:0x99b22c5bd258b3a7,NaT,None,NaN,2021
8,1.155862e+20,Janice Powers Johnson,2021-06-23 19:51:42.312,5,Professional helpful and friendly great custom...,None,0x88c33f1c8e1f99a3:0x99b22c5bd258b3a7,NaT,None,NaN,2021
9,1.102443e+20,SaSa Armor,2017-02-03 19:36:37.552,1,You are better off walking your item to it's d...,None,0x88c33f1c8e1f99a3:0x99b22c5bd258b3a7,NaT,None,NaN,2017
10,1.079347e+20,Jayde Machado,2017-12-30 01:20:02.183,5,I’ve always had a great experience shipping th...,None,0x88c33f1c8e1f99a3:0x99b22c5bd258b3a7,NaT,None,NaN,2017
11,1.109099e+20,Agnes Gustek,2014-09-24 17:06:58.729,1,We've sent extremely urgent International pack...,None,0x88c33f1c8e1f99a3:0x99b22c5bd258b3a7,NaT,None,NaN,2014
...,...,...,...,...,...,...,...,...,...,...,...
3825005,1.180458e+20,NaN,2018-10-18 13:22:50.951,3,None,None,0x89c3e2ffb6d69839:0xd8cffcad11213073,NaT,None,Jessica Austin Higgins,2018
3825006,1.094347e+20,NaN,2019-09-05 22:16:42.078,5,None,None,0x89c3e2ffb6d69839:0xd8cffcad11213073,2019-09-06 19:46:19.800999936,The we are so happy to hear from you! Thank y...,Aurora,2019
3825007,1.111502e+20,NaN,2020-10-08 23:03:34.610,5,None,None,0x89c3e2ffb6d69839:0xd8cffcad11213073,NaT,None,Kellie Zottman,2020
3825008,1.161815e+20,NaN,2020-02-09 14:53:56.442,4,None,None,0x89c3e2ffb6d69839:0xd8cffcad11213073,2020-02-09 23:56:14.768999936,Lawrence we are so happy to hear from you! Th...,Lawrence Chiappetta,2020
